# Introduction to Climate Sensitivity Constraint Framework

This notebook provides an introduction to the multi-constraint framework for estimating equilibrium climate sensitivity (ECS).

## Overview

**Equilibrium Climate Sensitivity (ECS)** is defined as the long-term global mean surface temperature change following a doubling of atmospheric CO₂ concentration.

This framework combines multiple lines of evidence:
1. **Paleoclimate constraints** (LGM, Pliocene, Last Interglacial)
2. **Observational constraints** (historical temperature, energy budget)
3. **Process-based constraints** (cloud feedbacks, regional patterns)

## Setup

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Setup complete!")

## Import Constraint Modules

In [ ]:
# Import constraint classes
from constraints.paleoclimate import LGMConstraint, PlioceneConstraint
from constraints.observational import EnergyBudgetConstraint, EmergentConstraint
from constraints.process_based import CloudFeedbackConstraint

# Import uncertainty quantification
from uncertainty.bayesian_integration import BayesianIntegrator

# Import validation
from validation.perfect_model import PerfectModelValidator

print("Modules imported successfully!")

## Example 1: Last Glacial Maximum Constraint

The LGM (~21,000 years ago) was approximately 5-7°C colder than pre-industrial.
We can use this to constrain climate sensitivity.

In [ ]:
# Initialize LGM constraint
lgm = LGMConstraint(apply_state_dependence=True, n_samples=10000)

# Typical values from Tierney et al. (2020) Nature
lgm_temp_change = -6.1  # K (cooling)
lgm_temp_unc = 0.4  # K

# Forcing from CO2, ice sheets, vegetation, etc.
lgm_forcing = -7.5  # W/m²
lgm_forcing_unc = 1.5  # W/m²

# Estimate ECS
lgm_result = lgm.estimate_ecs(
    temperature_change=lgm_temp_change,
    temperature_uncertainty=lgm_temp_unc,
    forcing_change=lgm_forcing,
    forcing_uncertainty=lgm_forcing_unc,
    correlation=-0.2
)

print(f"LGM Constraint Results:")
print(f"  ECS: {lgm_result.ecs_mean:.2f} ± {lgm_result.ecs_std:.2f} K")
print(f"  State-dependence factor: {lgm_result.state_dependence_factor:.2f}")
print(f"  Number of samples: {len(lgm_result.ecs_samples)}")

In [ ]:
# Plot LGM constraint distribution
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.hist(lgm_result.ecs_samples, bins=50, density=True, alpha=0.7, 
        edgecolor='black', label='LGM constraint')
ax.axvline(lgm_result.ecs_mean, color='red', linestyle='--', 
           linewidth=2, label=f'Mean: {lgm_result.ecs_mean:.2f} K')
ax.axvline(np.percentile(lgm_result.ecs_samples, 5), color='blue', 
           linestyle=':', label='5th-95th percentile')
ax.axvline(np.percentile(lgm_result.ecs_samples, 95), color='blue', 
           linestyle=':')

ax.set_xlabel('Equilibrium Climate Sensitivity (K)', fontsize=14)
ax.set_ylabel('Probability Density', fontsize=14)
ax.set_title('LGM Constraint on Climate Sensitivity', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Example 2: Energy Budget Constraint

Using historical warming and radiative forcing to constrain ECS.

In [ ]:
# Initialize energy budget constraint
energy_budget = EnergyBudgetConstraint(
    time_period=(1850, 2020),
    n_samples=10000
)

# Use default historical values (IPCC AR6)
eb_result = energy_budget.estimate_from_historical_data()

print(f"Energy Budget Constraint Results:")
print(f"  ECS: {eb_result.ecs_mean:.2f} ± {eb_result.ecs_std:.2f} K")
print(f"  5th-95th percentile: [{eb_result.ecs_5th_percentile:.2f}, {eb_result.ecs_95th_percentile:.2f}] K")
print(f"  Forcing used: {eb_result.effective_radiative_forcing:.2f} W/m²")

## Example 3: Combining Multiple Constraints

Using Bayesian integration to combine different lines of evidence.

In [ ]:
# Combine constraints using Bayesian integration
integrator = BayesianIntegrator(
    prior_type='jeffreys',
    n_samples=50000
)

# Prepare constraint data
constraint_data = {
    'LGM': lgm_result.ecs_samples,
    'Energy_Budget': eb_result.ecs_samples
}

# Integrate constraints
posterior = integrator.integrate_constraints(constraint_data)

print(f"\nCombined Posterior Results:")
print(f"  ECS: {posterior.ecs_mean:.2f} ± {posterior.ecs_std:.2f} K")
print(f"  Median: {posterior.ecs_median:.2f} K")
print(f"  90% CI: [{posterior.ecs_percentiles['5th']:.2f}, {posterior.ecs_percentiles['95th']:.2f}] K")
print(f"  Information gain: {posterior.information_gain:.2f} bits")

In [ ]:
# Compare all constraints
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

# Plot each constraint
ax.hist(lgm_result.ecs_samples, bins=40, density=True, alpha=0.5, 
        label='LGM', edgecolor='black')
ax.hist(eb_result.ecs_samples, bins=40, density=True, alpha=0.5, 
        label='Energy Budget', edgecolor='black')
ax.hist(posterior.ecs_samples, bins=40, density=True, alpha=0.7, 
        label='Combined', edgecolor='black', color='purple')

# Add IPCC AR6 likely range
ax.axvspan(2.5, 4.0, alpha=0.2, color='gray', label='IPCC AR6 likely range')

ax.set_xlabel('Equilibrium Climate Sensitivity (K)', fontsize=14)
ax.set_ylabel('Probability Density', fontsize=14)
ax.set_title('Comparison of Climate Sensitivity Constraints', fontsize=16)
ax.legend(fontsize=12, loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 8)

plt.tight_layout()
plt.show()

## Next Steps

Continue with:
- `02_individual_constraints.ipynb`: Detailed analysis of each constraint type
- `03_bayesian_integration.ipynb`: Advanced Bayesian methods
- `04_validation_analysis.ipynb`: Perfect model validation
- `05_results_visualization.ipynb`: Comprehensive results and figures

## References

- IPCC (2021). Climate Change 2021: The Physical Science Basis.
- Sherwood et al. (2020). An assessment of Earth's climate sensitivity. *Reviews of Geophysics*.
- Tierney et al. (2020). Glacial cooling and climate sensitivity. *Nature*.